# Классификация отзывов к фильмам. Бинарная классификация

## Используемые библиотеки

In [1]:
from keras.datasets import imdb
from keras import models
from keras import layers
import numpy as np
import matplotlib.pyplot as plt

## Загрузка данных

In [2]:
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=10000)

## Осмотр данных

In [3]:
word_index = imdb.get_word_index()
invert_word_index = {word_index.get(key): key for key in word_index.keys()}

In [4]:
review = ' '.join([invert_word_index.get(idx - 3, '?') for idx in x_train[0]])
print(review)

## Подготовка данных

In [5]:
def vectorize_sequences(sequences, dimension = 10_000):
    result = np.zeros((len(sequences), dimension))
    
    for i in range(len(sequences)):
        result[i, sequences[i]] = 1
        
    return result

In [6]:
x_train = vectorize_sequences(x_train)
x_test = vectorize_sequences(x_test)

In [7]:
y_train = y_train.astype('float32')
y_test = y_test.astype('float32')

##  Обучение модели

### Тренировочные и валидационные данные

In [8]:
x_val = x_train[:10_000]
partial_x_train = x_train[10_000:]

y_val = y_train[:10_000]
partial_y_train = y_train[10_000:]

### Конструирование сети

In [9]:
model = models.Sequential()

model.add(layers.Dense(16, activation='tanh', input_shape=(10_000,)))
model.add(layers.Dense(16, activation='tanh'))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])

### Обучение

In [10]:
history = model.fit(
    partial_x_train,
    partial_y_train,
    epochs=20,
    batch_size=512,
    validation_data=(x_val, y_val)
)

### Провека метрик

In [11]:
history_dict = history.history
history_dict.keys()

In [12]:
loss = history_dict['loss']
val_loss = history_dict['val_loss']

In [13]:
epochs = range(1, len(loss) + 1)

plt.plot(epochs, loss, label='Training loss')
plt.plot(epochs, val_loss, label='Validation loss')
plt.title('Training and validation data')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [14]:
acc = history_dict['acc']
val_acc = history_dict['val_acc']

In [15]:
plt.plot(epochs, acc, label='Training accuracy')
plt.plot(epochs, val_acc, label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

### Финальное обучение

In [16]:
model = models.Sequential()

model.add(layers.Dense(16, activation='relu', input_shape=(10_000,)))
model.add(layers.Dense(16, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])

In [17]:
history = model.fit(
    partial_x_train,
    partial_y_train,
    epochs=4,
    batch_size=512,
    validation_data=(x_val, y_val)
)

## Результат

In [18]:
result = model.evaluate(x_test, y_test)

In [19]:
result

## Выводы

От выбора количества нейронных слоев и количества нейронов в них зависит:
1. Точность модели
2. Время, через которое наступает переобучение

Поэтому очень важно найти правильный баланс. Необходимо не давать сети большой возможности по "запоминанию" данных с одной стороны, а с другой не создавать в сети узких мест, где будет терятся какое-то количество информации. Сеть должна учиться обобщать зависимости между данным, а не запоминать сами данные

[Продолжение](./%D0%9A%D0%BB%D0%B0%D1%81%D1%81%D0%B8%D1%84%D0%B8%D0%BA%D0%B0%D1%86%D0%B8%D1%8F%20%D0%BD%D0%BE%D0%B2%D0%BE%D1%81%D1%82%D0%BD%D1%8B%D1%85%20%D0%BB%D0%B5%D0%BD%D1%82.ipynb)